# Execute all the cells below

In [10]:
import math
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.chrome.options import Options
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import validators

In [11]:
#this function finds all the category which can be used for collecting the datas
def get_the_category_urls(url_having_all_categories):
    #validate whether the url is present or not
    if not validators.url(url_having_all_categories):
        return None, None, 1
    url = url_having_all_categories
    max_retries = 10

    for attempt in range(1, max_retries + 1):
        try:
            response = requests.get(url)
            response.raise_for_status() # Raises HTTPError for bad responses
            soup = BeautifulSoup(response.text, 'html.parser')
#             print("Title:", soup.title.text)
            break
        except requests.RequestException as e:
#             print(f"Attempt {attempt}: Request failed - {e}")
            pass

    else:
#         print(f"Failed to retrieve the content after {max_retries} attempts.")
        return None, None, 1

    
    all_hrefs = [a.get('href') for a in soup.find_all('a', class_='catHead')]
    all_hrefs=  [url[:-12]+i for i in all_hrefs]
    
    category_elements = soup.find_all('a', class_='catHead')
    category_names= [element.text for element in category_elements]
    
    return all_hrefs,category_names, 0


In [12]:
#what this function do is go inside all the category url and finds all the product's url
#we return three thing
#1.list_subcategories->having the urls for the product
#2.error_logs- lets say if accessing the given_url is done, we can find all the pages and subcategories which failed to load
#3.error-if the website fails to load, we can simply return the message 1
def find_all_urls(given_url):
    error_logs=[]
    list_subcategories=[]
    url_k=given_url

    subcategory_names=[]

    # url = "https://www.hippostores.com/ihbcategory/sanitary-ware-and-bath-fittings-sbf"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
    }

    
    #validate whether the url exist or not
    if not validators.url(url_k):
        return list_subcategories, subcategory_names, error_logs, 1
    
    #prevent from not getting a responce and getting a bad responce
    maxtries=5
    error=1
    for tries in range(maxtries):
        
        session = requests.Session()
        response = session.get(url_k, headers=headers)
        if response.status_code == 200 and response.status_code != 400:
            soup = BeautifulSoup(response.text, 'html.parser')
            error=0
            break

    if error==1:
        return list_subcategories, subcategory_names, error_logs, 1
    
    #first we have to find the name of the category
    
    target_elements = soup.find_all('p', {'class': 'productTitle'})

    for element in target_elements:
        product_title = element.find('a').contents[0].strip()
        subcategory_names.append(product_title)
    

    #list_category contains all the links for the subcategories in the specific category
    list_category=[]
    cat_colm_elements = soup.find_all(class_="cat-colm")


    # Extract and print the first link from each element
    for cat_colm in cat_colm_elements:
        href_link = "https://www.industrybuying.com"+cat_colm.find('a')['href']
        list_category.append(href_link)

    #so we get our list category which contains all the links for the subcategories in the specific category



    
    for i in range(len(list_category)):
        error_logs.append([])
        #this contains all the urls of that specific subcategory
        list_of_url_individual=[]

        #link_one_category is specific url of a subcategory
        link_one_category=list_category[i]
        error_logs[-1].append(link_one_category)
        headers = {
          'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
        }
        
        
        maxtries=5
        error=1
        for tries in range(maxtries):
            session = requests.Session()
            response = session.get(link_one_category, headers=headers)

            #prevent from not getting a responce and getting a bad responce
            if response.status_code == 200 and response.status_code != 400:
                soup = BeautifulSoup(response.text, 'html.parser')
                error=0
                error_logs[-1].append(0) #means this category is successfully captured
                error_logs[-1].append([])
                break
        if error==1:
#             error_logs[-1].append(1,[], 0, 0)
            error_logs[-1].append(1)
            error_logs[-1].append([])
            error_logs[-1].append(0)
            error_logs[-1].append(0)
            continue

        span_element = soup.find('span', class_='productslimit')

        span_text = span_element.get_text()

        span_text_parts = span_text.strip('()').split()

        products_one_page = int(span_text_parts[0].split('-')[1])
        total_products = int(span_text_parts[-1])
        pages=math.ceil(total_products / products_one_page)

        #now all we have to do is go page to page and pick all the url having the links
        for page in range(pages):
            url_page_wise=link_one_category+ f"?page={page+1}"

            headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
            }
            
            maxtries=5
            error=1
            for tries in range(maxtries):
                session = requests.Session()
                response = session.get(url_page_wise, headers=headers)
                #prevent from not getting a responce and getting a bad responce
                if response.status_code == 200 and response.status_code != 400:
                    soup = BeautifulSoup(response.text, 'html.parser')
                    error=0
                    break
                if error==1:
                    error_logs[-1][-1].append(page)
                    continue
            
                

            div_elements = soup.find_all('div', class_='AH_ProductView')



            # Loop through each div element
            for div_element in div_elements:
                # Find the first anchor element within the div
                first_link = div_element.find('a')

                # If a link is found, append its href to the list
                if first_link:
                    href_link = first_link.get('href')
                    list_of_url_individual.append(href_link)
        
        error_logs[-1].append(len(list_of_url_individual))
        list_subcategories.append(list_of_url_individual)
        error_logs[-1].append(len(list(set(list_of_url_individual))))
#         print(i, end=" ")
    
    #remove duplicate from the list_subcategories
    list_subcategories_copy=[]
    for lis in list_subcategories:
        lis_set=list(set(lis))
        list_subcategories_copy.append(lis_set)
    
    list_subcategories=list_subcategories_copy
    #we can return this table without any duplicate index
    
    
    return list_subcategories, subcategory_names, error_logs, 0


In [13]:
def logs_table(logs, category_name, subcategory_names, category_url):
    for log in logs:
        log[2] = [str(num) for num in log[2]]
        lis=log[2]
        log[2]=', '.join(lis)
        
    columns=['Category URL', 'Sub Category URL', 'Category', 'Sub Category', 'Category Check', 'Sub Category Check', 'Pages Check', 'Page Not Retrieved', 'Total Product', 'Non duplicated Products']
        
    
    df=pd.DataFrame(logs, columns=['Sub Category URL','Sub Category Check','Page Not Retrieved','Total Product','Non duplicated Products'])
    df['Category URL']=category_url
    df['Category']=category_name
    df['Sub Category']=subcategory_names
    df['Category Check']=0
    df['Pages Check'] = df['Page Not Retrieved'].apply(lambda x: 0 if x == '' else 1)
    
    df=df[columns]
    
    filename="IndustryBuying"+"__"+category_name+"__"+"logs.csv"
    df.to_csv(filename, index=False)
    return logs

In [14]:
#what it does is find all the prices inside the table listing the mrp and other stuff in each product
def find_prices(soup):
    mrp=None
    discount=None
    discount_flag=None
    
    price_text = soup.find('span', class_='mainPrice')
    
    
    if price_text:
        price_text=price_text.text
        price_integer = int(price_text.replace('Rs.', '').replace(',', ''))
        mrp=price_integer
        discount_flag=0
        discount="1-"+str(mrp)
        return mrp, discount, discount_flag, 0
    
    mrp_listing = soup.find('del', {'id': 'AH_ListPrice'})
    bulk_table= soup.find('div', {'class': 'ah-bulk-qty-table'})
    per_piece_price=soup.find('span', class_='AH_PricePerPiece')
    
    if mrp_listing is None and bulk_table is None and per_piece_price is None:
        return None, None, None, 1

    
    
    #find a condition with no discount on individual as well as on packs
    if mrp_listing is None and bulk_table is None:
        discount_flag=0
        #now since there wont be any discount either ways we can have our discount as:- mrp-1
        mrp=per_piece_price.get_text(strip=True).replace(",", "")
        discount="1-"+str(mrp)
        
    
    #find a condition with discount only on 1 item 
    elif mrp_listing and bulk_table is None:
        discount_flag=1
        #now my discount price and my mrp would be different
        discount=per_piece_price.get_text(strip=True).replace(",", "")
        discount="1-"+str(discount)
        mrp=mrp_listing.get_text(strip=True).replace(",", "")
        
        
    #find a condition where discount is only present on bulk
    elif mrp_listing is None and bulk_table:
        discount_flag=2
        #now since there wont be any discount(individual) we can have our discount initially as:- mrp-1
        mrp=per_piece_price.get_text(strip=True).replace(",", "")
        discount="1-"+str(mrp)
        
        element_texts = []

        # Start the index at 0
        index = 0

        while True:
            # Generate the class name with the current index
            class_name = f'table-row bulk-option AH_BulkPriceOption AH_BulkPriceOption_{index}'

            # Find all div elements with the specific class
            elements = soup.find_all('div', class_=class_name)

            # If elements are found, proceed
            if elements:
                for element in elements:
                    # Find all div elements with class "table-cell" inside the current element
                    table_cells = element.find_all('div', class_='table-cell')

                    # Extract text from each "table-cell" and add it to the list
                    cell_texts = [cell.get_text(strip=True) for cell in table_cells]
                    element_texts.append(cell_texts)

                # Increment the index for the next iteration
                index += 1
            else:
                # If no elements are found, break the loop
                break

        # Print the list of texts for each element
        for each_element in element_texts:
            a=each_element[1]
            b=each_element[2]
            a=a.split('-')[0].strip().split('+')[0]
            b=b.split(" ")[-1]
            #now we got these dudes, we can simple append this to string discount
            discount+="; "+a+"-"+b
        
    
    #find a condition where the discount is present on both, individual and bulk
    else:
        discount_flag=3
        #now we can different discount for individual and bulk
        discount=per_piece_price.get_text(strip=True).replace(",", "")
        discount="1-"+str(discount)
        mrp=mrp_listing.get_text(strip=True).replace(",", "") 
        
        element_texts = []

        # Start the index at 0
        index = 0

        while True:
            # Generate the class name with the current index
            class_name = f'table-row bulk-option AH_BulkPriceOption AH_BulkPriceOption_{index}'

            # Find all div elements with the specific class
            elements = soup.find_all('div', class_=class_name)

            # If elements are found, proceed
            if elements:
                for element in elements:
                    # Find all div elements with class "table-cell" inside the current element
                    table_cells = element.find_all('div', class_='table-cell')

                    # Extract text from each "table-cell" and add it to the list
                    cell_texts = [cell.get_text(strip=True) for cell in table_cells]
                    element_texts.append(cell_texts)

                # Increment the index for the next iteration
                index += 1
            else:
                # If no elements are found, break the loop
                break

        # Print the list of texts for each element
        for each_element in element_texts:
            a=each_element[1]
            b=each_element[2]
            a=a.split('-')[0].strip().split('+')[0]
            b=b.split(" ")[-1]
            #now we got these dudes, we can simple append this to string discount
            discount+="; "+a+"-"+b
        
        
        
    
    #our discount will be in integer but our mrp must be integer
    mrp=int(mrp)
    
    
    return mrp, discount, discount_flag, 0

In [15]:
#these function can be used for cleaning
#for eg:-name of the product is:-Super Bulb with ultra shining(332743), then my name_cleaner outputs:-Super Bulb with ultra shining
def name_cleaner(str):
  l=str.split(' ')
  clean_str=""
  if l[-1].isnumeric():
    clean_str=' '.join(l[0:-2])
  elif l[-1][1:-1].isnumeric():
    clean_str=' '.join(l[0:-1])
  else:
    clean_str=str

  return clean_str

print(name_cleaner("Today is an important match - 201920"))
print(name_cleaner("Today is an important match (38929)"))
print(name_cleaner("Today is an important match"))

#string star remover and capitalizer
def process_string(input_string):
    cleaned_string = input_string.rstrip('.*').lower().capitalize()
    return cleaned_string

print(process_string("health."), process_string("health*"), process_string("Philips bulbs"))

Today is an important match
Today is an important match
Today is an important match
Health Health Philips bulbs


In [16]:
#once we send our both list through which we can make the dataset
def get_dataset(list_mandatory, list_optional, set_with_columns, category_name):
    
    columns = ["Primary Key", "Brand", "L1", "L2", "L3", "L4", "L5", "L6", "Product",
               "Name", "Piece Quantity", "MRP", "Discount Price(Max)", "Discount Price(Min)",
               "Discount Check Flag", "URL"]

    
    # Create a DataFrame
    
    df_mandatory = pd.DataFrame(list_mandatory, columns=columns)   
    filename="IndustryBuying"+"__"+category_name+"__"+"MANDATORY.csv"
    df_mandatory.to_csv(filename,index=False)
    #we have made the mandatory dataset
    
    data = []

    # Iterate through each row in list_optional
    for row in list_optional:
        # Create a dictionary to store values for the current row
        row_values = {column: None for column in set_with_columns}

        # Update the values for the columns present in the current row
        for column, value in row:
            row_values[column] = value

        # Append the row to the list
        data.append(row_values)

    # Convert the list of dictionaries to a DataFrame
    df_optional = pd.DataFrame(data)
    filename="IndustryBuying"+"__"+category_name+"__"+"OPTIONAL.csv"
    df_optional.to_csv(filename, index=False)
    #we have made the optional dataset
    df=df_optional.copy()
    # Sort columns based on the number of NaN values in each column
    sorted_columns = df_optional.isna().sum().sort_values().index
    df = df[sorted_columns]

    df.to_csv(filename, index=False)
    

    return None

In [17]:
def data_collect(list_subcategories, category_name):
    list_mandatory=[]
    product_with_inconsistency=[]
    set_with_columns=set()
    list_optional=[]
    for i in range(len(list_subcategories)):
        for j in range(1):
    #         url="https://www.industrybuying.com/knapsack-sprayer-agripro-AGR.KNA.45809501/"

            url="https://www.industrybuying.com"+list_subcategories[i][j]


            options = Options()
            options.add_argument('--headless')

            try:
                # Attempt to open the Chrome WebDriver
                driver = webdriver.Chrome(options=options)

                # Attempt to navigate to the given URL
                driver.get(url)

            except Exception as e:
                # Handle the exception (e.g., print an error message)
                product_with_inconsistency.append([url, "URL inconsistency"])
                # Optionally, you may want to quit the driver if an exception occurs
                driver.quit()
#                 print(f"**->{len(product_with_inconsistency)}")
                continue




            try:
                button = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="main"]/div/div/div/div/div/div[1]/div[4]/div[2]/div[7]/div[1]/table/tbody/tr[8]/td/a')))
                if button:
                    button.click()
            except Exception as e:
                pass


            # time.sleep(1)

            try:
                # Get the latest HTML content
                html_content = driver.page_source
            except Exception as e:
                product_with_inconsistency.append([url, "URL inconsistency"])
                driver.quit()
#                 print(f"**->{len(product_with_inconsistency)}")
                continue


            # Use BeautifulSoup to parse the HTML content
            soup = BeautifulSoup(html_content, 'html.parser')



            #first we have to get the mandotory fields
            #it contains:- Primary Key, Brand, Category, Sub Category, Hidden Categories, Product, Name
            #MRP, Discount Price, Discount Check Flag, URL


            #lets figure out the prices and the discounts and discount flag
            mrp, discount, discount_flag, error=find_prices(soup)

            if error==1:
                product_with_inconsistency.append([url, "Bad Gateway inconsistency"])
#                 print(f"Bad Gateway ->{len(product_with_inconsistency)}")
                continue

            discount_max=discount.split(";")[0].split("-")[1]
            discount_min=discount.split("; ")[-1].split("-")[-1]


            #lets get the brand
            h2_element = soup.find('h2', class_='by')

            if h2_element:
                a_tag = h2_element.find('a')

                if a_tag:
                    brand=a_tag.text.strip()


            #now we get the category, subcategory, name and the product
            links = soup.select('div.commonBreadCrums a')

            # Extract the text inside each link and store it in a list
            links_list = [link.text.strip() for link in links]

            #if we found out the length of the links_list is less than 4, we will continue
            if len(links_list)<3:
                product_with_inconsistency.append([url, "Category listing inconsistency"])
#                 print(f"**->{len(product_with_inconsistency)}")
                continue

            
            #we have got all the subcategory using this
            l=links_list.copy()
            dict_cat={}
            for z in range(6):
                if len(l)-2>z and z!=5:
                    dict_cat[f"l{z+1}"]=l[z+1]
                else:
                    dict_cat[f"l{z+1}"]=""

                if z==5 and len(l)-2>z:
                    dict_cat[f"l{z+1}"]=" ".join([v for v in l[z+1:-1]])       
    
            product=l[-1]

            #now we will have some constants
            piece_quantity=1


            #now we have to get the name
            name = soup.find('span', class_='productTitle').find('h1').text.strip()


            #first of all we would be scrapping is optional stuff
            dict_table={}

            table_element = soup.find('div', {'class': 'tabDetailsContainer', 'id': 'famSpec'})


            # Find all div elements with class 'filterRow'
            if table_element:
                filter_rows = table_element.find_all('div', class_='filterRow')
            else:
                filter_rows=None
            # Initialize an empty list to store the tuple pairs
            dict_table={}



            # Loop through each filter row and extract feature name and value
            if filter_rows:
                for row in filter_rows:
                    key = row.find('div', class_='featureNamePr').text.strip()
                    key=process_string(key)
                    value = row.find('div', class_='featureValuePr').text.replace(':', '').strip()
                    #now we have got the feature_name and feature_value
                    #we can put this in a dictionary
                    dict_table[key]=str(value)


            #lets consider a scenario, if the model no is not present, we wont be definitely moving forward 
            u_id=None
            if 'Model no' in dict_table:
                u_id=dict_table['Model no']
            else:
                #if model_no not present, then put the id format as:-ibyymmddhhmmssuuuuuu (u denotes microseconds)
                current_time = datetime.now()
                # Format the time to "yymmddhhmmssuuuuuu"
                formatted_time = current_time.strftime("%y%m%d%H%M%S%f")[:]  
                u_id="ib"+formatted_time

            if u_id =='-':
                #if model_no not present, then put the id format as:-ibyymmddhhmmssuuuuuu (u denotes microseconds)
                current_time = datetime.now()
                # Format the time to "yymmddhhmmssuuuuuu"
                formatted_time = current_time.strftime("%y%m%d%H%M%S%f")[:]  
                u_id="ib"+formatted_time
                
            dict_table['Model no']=u_id 

            #now we can add all the optional columns in the set
            for k in dict_table:
                set_with_columns.add(k)


            key_value_pairs_as_lists = [[key, value] for key, value in dict_table.items()]
            list_optional.append(key_value_pairs_as_lists)
            print(i*2+j+1)

    #         print(brand)

    #         print(category, subcategory, product, name, sep='\t')

    #         print(u_id, discount_flag, mrp, discount_max, discount_min, sep='\t')

            #Primary Key, Brand, Category, Sub Category, Hidden Categories, Product, Name, Piece Quantity, MRP, Discount Price(Max), Discount Price(Min), Discount Check Flag, URL
            list_mandatory.append([u_id, brand, dict_cat["l1"], dict_cat["l2"], dict_cat["l3"], dict_cat["l4"], dict_cat["l5"], dict_cat["l6"], product, name, piece_quantity, mrp, discount_max, discount_min, discount_flag, url])
#             print(len(list_mandatory))
            
    #now since we got two things, first is the mandatory list and second is the optional list,
    #we can get the dataset from both of them
    get_dataset(list_mandatory, list_optional, set_with_columns, category_name)
    #what this function does is
    

    return product_with_inconsistency

In [20]:
url_having_all_categories="https://www.industrybuying.com/categories/"

urls,category_names, error=get_the_category_urls(url_having_all_categories)

if error==1:
    print("There is problem in the url having the categories")
    sys.exit()

# urls=['https://www.industrybuying.com/agriculture-garden-landscaping-2384/', 'unknown', 'https://www.industrybuying.com/pumps-1160/', 'https://www.industrybuying.com/solar-4050/', 'please go away', 'https://www.industrybuying.com/welding-552/']
    
for index, i_th_url in enumerate(urls):
    #implement this function and get all the url
    ith_category_name=category_names[index]
    list_subcategories, subcategory_names, logs, error=find_all_urls(i_th_url)
    
    print(f"The structure of the urls looks like:- (urls-{i_th_url})")
    for lis in list_subcategories:
        print(len(lis), end=" ")
    print()
    
    #we have got the logs and error
    #if the error ==1, we can simply return the dataframe having the heading as failed 
    if error==1:
        data_logs = {'Category URL':[i_th_url], 'Sub Category URL':[None], 'Category':[ith_category_name], 'Sub Category':[None], 
                     'Category Check':[1], 'Sub Category Check': [0], 'Pages Check':[0], 'Page Not Retrieved':[""],'Total Product':[0],'Non duplicated Products':[0]}
        df = pd.DataFrame(data_logs)
        filename="IndustryBuying"+"__"+ith_category_name+"__"+"logs.csv"
        df.to_csv(filename, index=False)
        #since the data_logs has no value in it, we can dont have the need to define the mandatory, optional and inconsistent urls
        continue
    #we must define the logs
    #since we got the logs, we can simply find the dataframe
    logs_table(logs, ith_category_name, subcategory_names, i_th_url)
    
    
    product_with_inconsistency=data_collect(list_subcategories, ith_category_name)
    df = pd.DataFrame(product_with_inconsistency, columns=['URL', 'ISSUE'])
    df['Category']=ith_category_name
    df=df[['Category', 'URL', 'ISSUE']]
    filename="IndustryBuying"+"__"+ith_category_name+"__"+"InconsistentProducts.csv"
    df.to_csv(filename, index=False)
        
    #now all we need to do is get the links which have inconsistency


The structure of the urls looks like:- (urls-https://www.industrybuying.com/abrasives-642/)
1500 1321 1459 271 359 338 349 682 436 187 996 
1
3
5
7
9
11
13
15
17
19
21
The structure of the urls looks like:- (urls-https://www.industrybuying.com/adhesives-sealants-and-tape-2595/)
649 7030 361 2857 1308 889 83 172 188 231 342 135 51 60 195 
1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
The structure of the urls looks like:- (urls-https://www.industrybuying.com/agriculture-garden-landscaping-2384/)
1289 834 1123 48 253 48 339 578 223 110 1283 369 71 94 80 252 67 853 3 
1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
The structure of the urls looks like:- (urls-https://www.industrybuying.com/appliances-17567/)
2642 150 1309 1135 462 142 
1
3
5
7
9
11


KeyboardInterrupt: 

In [28]:
import pandas as pd
import os
from random import randint


directory = os.getcwd()  

#lets build dataset for the inconsistent products
csv_files = [file for file in os.listdir(directory) if file.endswith("InconsistentProducts.csv")]


combined_df = pd.DataFrame()

for csv_file in csv_files:
    file_path = os.path.join(directory, csv_file)
    df = pd.read_csv(file_path)
    combined_df = pd.concat([df, combined_df])
combined_df_reset = combined_df.reset_index(drop=True)




directory = os.path.join(os.getcwd(), 'overall_logs')
file_path = os.path.join(directory, 'summarised_InconsistentProducts.csv')

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the DataFrame to a CSV file
combined_df_reset.to_csv(file_path, index=False)


#lets do the same for logs
# Define the directory where the CSV files are located
directory = os.getcwd()  

# Get a list of CSV files ending with "InconsistentProducts" in the directory
csv_files = [file for file in os.listdir(directory) if file.endswith("logs.csv")]


combined_df = pd.DataFrame()
# Loop through each CSV file and append its data to the combined DataFrame
for csv_file in csv_files:
    file_path = os.path.join(directory, csv_file)
    df = pd.read_csv(file_path)
    combined_df = pd.concat([df, combined_df])
combined_df_reset = combined_df.reset_index(drop=True)
combined_df_reset




directory = os.path.join(os.getcwd(), 'overall_logs')
file_path = os.path.join(directory, 'summarised_logs.csv')

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the DataFrame to a CSV file
combined_df_reset.to_csv(file_path, index=False)



# The cell below consist of testing code.
# These cells wont be utitlized

In [ ]:
# # url_having_all_categories="https://www.industrybuying.com/categories/"

# # urls,category_names, error=get_the_category_urls(url_having_all_categories)

# # if error==1:
# #     print("There is problem in the url having the categories")
# #     sys.exit()

# urls=['nahi hai']
# category_names=['umhmmm']
# for index, i_th_url in enumerate(urls):
#     #implement this function and get all the url
#     ith_category_name=category_names[index]
#     list_subcategories, subcategory_names, logs, error=find_all_urls(i_th_url)
    
#     print(f"The structure of the urls looks like:- (urls-{i_th_url})")
#     for lis in list_subcategories:
#         print(len(lis), end=" ")
#     print()
    
#     #we have got the logs and error
#     #if the error ==1, we can simply return the dataframe having the heading as failed 
#     if error==1:
#         data_logs = {'Category URL':[i_th_url], 'Sub Category URL':[None], 'Category':[ith_category_name], 'Sub Category':[None], 
#                      'Category Check':[1], 'Sub Category Check': [0], 'Pages Check':[0], 'Page Not Retrieved':[""],'Total Product':[0],'Non duplicated Products':[0]}
#         df = pd.DataFrame(data_logs)
#         filename="IndustryBuying"+"__"+ith_category_name+"__"+"logs.csv"
#         df.to_csv(filename, index=False)
#         #since the data_logs has no value in it, we can dont have the need to define the mandatory, optional and inconsistent urls
#         continue
#     #we must define the logs
#     #since we got the logs, we can simply find the dataframe
#     logs_table(logs, ith_category_name, subcategory_names, i_th_url)
    
    
#     product_with_inconsistency=data_collect(list_subcategories, ith_category_name)
#     df = pd.DataFrame(product_with_inconsistency, columns=['URL', 'ISSUE'])
#     df['Category']=ith_category_name
#     df=df[['Category', 'URL', 'ISSUE']]
#     filename="IndustryBuying"+"__"+ith_category_name+"__"+"InconsistentProducts.csv"
#     df.to_csv(filename, index=False)
        
#     #now all we need to do is get the links which have inconsistency


In [ ]:
# import pandas as pd
# import os
# from random import randint


# directory = os.getcwd()  

# #lets build dataset for the inconsistent products
# csv_files = [file for file in os.listdir(directory) if file.endswith("InconsistentProducts.csv")]


# combined_df = pd.DataFrame()

# for csv_file in csv_files:
#     file_path = os.path.join(directory, csv_file)
#     df = pd.read_csv(file_path)
#     combined_df = pd.concat([df, combined_df])
# combined_df_reset = combined_df.reset_index(drop=True)




# directory = os.path.join(os.getcwd(), 'overall_logs')
# file_path = os.path.join(directory, 'summarised_InconsistentProducts.csv')

# # Create the directory if it doesn't exist
# if not os.path.exists(directory):
#     os.makedirs(directory)

# # Save the DataFrame to a CSV file
# combined_df_reset.to_csv(file_path, index=False)


# #lets do the same for logs
# # Define the directory where the CSV files are located
# directory = os.getcwd()  

# # Get a list of CSV files ending with "InconsistentProducts" in the directory
# csv_files = [file for file in os.listdir(directory) if file.endswith("logs.csv")]


# combined_df = pd.DataFrame()
# # Loop through each CSV file and append its data to the combined DataFrame
# for csv_file in csv_files:
#     file_path = os.path.join(directory, csv_file)
#     df = pd.read_csv(file_path)
#     combined_df = pd.concat([df, combined_df])
# combined_df_reset = combined_df.reset_index(drop=True)
# combined_df_reset




# directory = os.path.join(os.getcwd(), 'overall_logs')
# file_path = os.path.join(directory, 'summarised_logs.csv')

# # Create the directory if it doesn't exist
# if not os.path.exists(directory):
#     os.makedirs(directory)

# # Save the DataFrame to a CSV file
# combined_df_reset.to_csv(file_path, index=False)



In [ ]:
# import pandas as pd
# import os

# # Define the directory where the CSV files are located
# directory = os.getcwd()  # You can change this to your desired directory

# # Get a list of CSV files ending with "InconsistentProducts" in the directory
# csv_files = [file for file in os.listdir(directory) if file.endswith("InconsistentProducts.csv")]


# combined_df = pd.DataFrame()
# # Loop through each CSV file and append its data to the combined DataFrame
# for csv_file in csv_files:
#     file_path = os.path.join(directory, csv_file)
#     df = pd.read_csv(file_path)
#     combined_df = pd.concat([df, combined_df])
# combined_df_reset = combined_df.reset_index(drop=True)
# combined_df_reset


In [ ]:
# import pandas as pd
# import os

# # Define the directory where the CSV files are located
# directory = os.getcwd()  # You can change this to your desired directory

# # Get a list of CSV files ending with "InconsistentProducts" in the directory
# csv_files = [file for file in os.listdir(directory) if file.endswith("logs.csv")]


# combined_df = pd.DataFrame()
# # Loop through each CSV file and append its data to the combined DataFrame
# for csv_file in csv_files:
#     file_path = os.path.join(directory, csv_file)
#     df = pd.read_csv(file_path)
#     combined_df = pd.concat([df, combined_df])
# combined_df_reset = combined_df.reset_index(drop=True)
# combined_df_reset
